# Explore Synthetic Telco Churn + Support Conversations Data

Цей ноутбук показує:
- базовий огляд табличних даних клієнтів
- візуалізацію concept/data drift (зміна churn rate та розподілів у часі)
- аналіз згенерованих support conversations
- зв'язок між типами скарг та churn

Дані знаходяться в директорії `../data/`

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style("whitegrid")

DATA_DIR = "../data/"

## 1. Завантаження даних

In [ ]:
df_customers = pd.read_csv(DATA_DIR + "telco_customers.csv")
df_customers["RecordDate"] = pd.to_datetime(df_customers["RecordDate"])

df_conversations = pd.read_csv(DATA_DIR + "support_conversations.csv")
df_conversations["RecordDate"] = pd.to_datetime(df_conversations["RecordDate"])

df_kb = pd.read_csv(DATA_DIR + "knowledge_base.csv")

print("Клієнти:          ", df_customers.shape)
print("Розмови support:  ", df_conversations.shape)
print("Knowledge base:   ", df_kb.shape)

## 2. Огляд табличних даних клієнтів

In [ ]:
df_customers.head(3)

In [ ]:
df_customers.info()

In [ ]:
df_customers.describe(include="all")

## 3. Аналіз churn drift у часі

In [ ]:
df_customers["Year"] = df_customers["RecordDate"].dt.year
df_customers["Month"] = df_customers["RecordDate"].dt.to_period("M")

churn_by_year = df_customers.groupby("Year")["Churn"].value_counts(normalize=True).unstack().round(3)
print("Churn rate по роках:\n", churn_by_year)

plt.figure(figsize=(10, 5))
churn_by_year["Yes"].plot(marker="o", label="Churn = Yes")
plt.title("Зміна churn rate у часі (drift demonstration)")
plt.ylabel("Частка клієнтів, що пішли")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Динаміка Fiber optic adoption
fiber_by_year = df_customers.groupby("Year")["InternetService"].value_counts(normalize=True).unstack().get("Fiber optic", pd.Series())

plt.figure(figsize=(10, 5))
fiber_by_year.plot(marker="o", color="darkgreen")
plt.title("Частка клієнтів з Fiber optic у часі")
plt.ylabel("Частка")
plt.grid(True)
plt.show()

## 4. Аналіз support conversations

In [ ]:
print("Розподіл типів проблем:")
print(df_conversations["issue_type"].value_counts(normalize=True).round(3) * 100, "%")

plt.figure(figsize=(10, 5))
sns.countplot(data=df_conversations, x="issue_type", order=df_conversations["issue_type"].value_counts().index, palette="Set2")
plt.title("Розподіл типів скарг")
plt.xticks(rotation=45)
plt.ylabel("Кількість скарг")
plt.show()

In [ ]:
print("Приклади розмов (випадкові):")
for _ in range(3):
    row = df_conversations.sample(1).iloc[0]
    print(f"\nТип: {row['issue_type'].upper()}")
    print(f"Скарга:   {row['complaint']}")
    print(f"Відповідь: {row['resolution']}")
    print("─" * 80)

## 5. Зв'язок між типом скарги та churn

In [ ]:
merged = df_conversations.merge(
    df_customers[["customerID", "Churn", "MonthlyCharges", "Contract", "InternetService"]],
    on="customerID",
    how="left"
)

print("Churn rate за типом скарги:")
print(merged.groupby("issue_type")["Churn"].value_counts(normalize=True).unstack().round(3))

plt.figure(figsize=(11, 6))
sns.countplot(data=merged, x="issue_type", hue="Churn", palette="coolwarm")
plt.title("Кількість скарг за типом та фактом відтоку")
plt.xticks(rotation=45)
plt.show()

## 6. Knowledge Base (для RAG / чат-агента)

In [ ]:
print("Knowledge Base документи:")
display(df_kb)

Готово! Дані можна використовувати для:
- тренування churn-прогнозу
- fine-tuning / RAG для чат-агента
- демонстрації data & concept drift